In [1]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split

In [2]:
# 데이터 불러오기
data = pd.read_csv("/Users/leejunseung/Desktop/머신러닝과 데이터분석 강의자료/Part 05~11) Machine Learning/08. Ensemble 기법의 종류와 원리/data/kc_house_data.csv") 
data.head() # 데이터 확인

,id,date,price,bedrooms,bathrooms,floors,waterfront,condition,grade,yr_built,yr_renovated,zipcode,lat,long
0,7129300520,20141013T000000,221900.0,3,1.00,1.0,0,3,7,1955,0,98178,47.5112,-122.257
1,6414100192,20141209T000000,538000.0,3,2.25,2.0,0,3,7,1951,1991,98125,47.7210,-122.319
2,5631500400,20150225T000000,180000.0,2,1.00,1.0,0,3,6,1933,0,98028,47.7379,-122.233
3,2487200875,20141209T000000,604000.0,4,3.00,1.0,0,5,7,1965,0,98136,47.5208,-122.393
4,1954400510,20150218T000000,510000.0,3,2.00,1.0,0,3,8,1987,0,98074,47.6168,-122.045


In [3]:
data = data.drop(['id','date','zipcode','lat','long'],axis=1)

In [4]:
feature_columns = list(data.columns.difference(['price']))
X = data[feature_columns]
y = data['price']
train_x, test_x, train_y, test_y = train_test_split(X,y,test_size=0.3,random_state=777)

In [5]:
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error , r2_score
from math import sqrt

sm_train_x = sm.add_constant(train_x,has_constant='add')
sm_model = sm.OLS(train_y,sm_train_x)
fitted_sm_model = sm_model.fit()
fitted_sm_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.601
Model:                            OLS   Adj. R-squared:                  0.600
Method:                 Least Squares   F-statistic:                     2843.
Date:                Mon, 03 May 2021   Prob (F-statistic):               0.00
Time:                        20:20:31   Log-Likelihood:            -2.0833e+05
No. Observations:               15129   AIC:                         4.167e+05
Df Residuals:                   15120   BIC:                         4.167e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         7.302e+06   1.73e+05     42.267      0.000    6.96e+06    7.64e+06
bathrooms     1.261e+05   3952.227     31.897      0.000    1.18e+05    1.34e+05
bedrooms     -1557.1160   2493.645     -0.624      0.532   -6444.962    3330.730
condition     1.956e+04   3179.808      6.150      0.000    1.33e+04    2.58e+04
floors        8884.3771   4348.553      2.043      0.041     360.687    1.74e+04
grade         1.958e+05   2206.118     88.750      0.000    1.91e+05       2e+05
waterfront    8.188e+05   2.15e+04     38.135      0.000    7.77e+05    8.61e+05
yr_built     -4367.7103     87.870    -49.707      0.000   -4539.945   -4195.475
yr_renovated     7.4940      5.037      1.488      0.137      -2.380      17.368
==============================================================================
Omnibus:                    12850.485   Durbin-Watson:                   1.978
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1284896.861
Skew:                           3.557   Prob(JB):                         0.00
Kurtosis:                      47.583   Cond. No.                     1.81e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.81e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [6]:
sm_test_x = sm.add_constant(test_x,has_constant='add')
sm_model_predict = fitted_sm_model.predict(sm_test_x)
print("RMSE: {}".format(sqrt(mean_squared_error(sm_model_predict, test_y)))) # RMSE
print(fitted_sm_model.params)

RMSE: 237325.2509669477
const           7.301951e+06
bathrooms       1.260647e+05
bedrooms       -1.557116e+03
condition       1.955544e+04
floors          8.884377e+03
grade           1.957922e+05
waterfront      8.187993e+05
yr_built       -4.367710e+03
yr_renovated    7.494013e+00
dtype: float64


In [14]:
bagging_predict_result = []
for i in range(10):
    data_index = [data_index for data_index in range(train_x.shape[0])]
    random_data_index = np.random.choice(data_index,train_x.shape[0])
    sm_train_x = train_x.iloc[random_data_index, ] # 랜덤 인덱스에 해당되는 학습 데이터 중 설명 변수
    sm_train_y = train_y.iloc[random_data_index, ] # 랜덤 인덱스에 해당되는 학습 데이터 중 종속 변수
    sm_train_x = sm.add_constant(sm_train_x,has_constant='add')
    sm_model = sm.OLS(sm_train_y,sm_train_x) #모델 구축
    fitted_sm_model = sm_model.fit()
    
    sm_test_x = sm.add_constant(test_x,has_constant='add')
    sm_model_predict = fitted_sm_model.predict(sm_test_x) #테스트 데이터 예측
    bagging_predict_result.append(sm_model_predict)

In [19]:
bagging_predict_result[0] #10개의 결과 생성 

18085    317150.085156
4749     355421.847516
7317     464546.279799
16406    420200.952639
17997    745585.919686
             ...      
19679    504184.057185
14433    322453.665912
12465    183475.394769
16320    368086.842510
5736     621292.568887
Length: 6484, dtype: float64

In [20]:
#Bagging을 바탕으로 예측한 결과값에 대한 평균 계산 
bagging_predict=[]
for lst2_index in range(test_x.shape[0]): #테스트 데이터 개수만큼 반복 
    temp_predict = [] #임시 빈 리스트 생성 (반복문 내 결과값 저장)
    for lst_index in range(len(bagging_predict_result)):
        temp_predict.append(bagging_predict_result[lst_index].values[lst2_index])
    bagging_predict.append(np.mean(temp_predict))    

In [21]:
# 예측한 결과값들의 평균을 계산하여 실제 테스트 데이트의 타겟변수와 비교하여 성능 평가
print("RMSE: {}".format(sqrt(mean_squared_error(bagging_predict, test_y)))) # RMSE

RMSE: 237307.77362129765


In [23]:
#학습 데이터를 선형회귀 모형에 적합 후 평가 데이터로 검증 
from sklearn.linear_model import LinearRegression
regression_model = LinearRegression() # 선형 회귀 모형
linear_model1 = regression_model.fit(train_x, train_y) # 학습 데이터를 선형 회귀 모형에 적합
predict1 = linear_model1.predict(test_x) # 학습된 선형 회귀 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict1, test_y)))) # RMSE 결과

RMSE: 237325.2509669477


In [25]:
#Bagging을 이용하여 선형 회귀 모형에 적합 후 평가 
from sklearn.ensemble import BaggingRegressor
bagging_model = BaggingRegressor(base_estimator = regression_model, # 선형회귀모형
                                 n_estimators = 5, # 5번 샘플링
                                 verbose = 1) # 학습 과정 표시
linear_model2 = bagging_model.fit(train_x, train_y) # 학습 진행
predict2 = linear_model2.predict(test_x) # 학습된 Bagging 선형 회귀 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict2, test_y)))) # RMSE 결과

RMSE: 237379.41105838222


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [26]:
#sampling up
bagging_model2 = BaggingRegressor(base_estimator = regression_model, # 선형 회귀모형
                                  n_estimators = 30, # 30번 샘플링
                                  verbose = 1) # 학습 과정 표시
linear_model3 = bagging_model2.fit(train_x, train_y) # 학습 진행
predict3 = linear_model3.predict(test_x) # 학습된 Bagging 선형 회귀 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict3, test_y)))) # RMSE 결과

RMSE: 237392.92626947773


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
